In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from slack import SlackInteractor
from claude_llm import ClaudeLLM
from project_manager_agent import ProjectManagerAgent
from sarcastic_agent import SarcasticAgent
from paul_graham_agent import PaulGrahamAgent  # Add this import
from db import ActionDatabase
from runner import process_threads, execute_due_actions
from config import CONFIG

In [11]:
workspaces = CONFIG['workspaces']
slack_interactors = {
    workspace_name: SlackInteractor(workspace_config)
    for workspace_name, workspace_config in workspaces.items()
}
llm = ClaudeLLM()
action_db = ActionDatabase()

agents = {
    workspace_name: [
        ProjectManagerAgent(llm, action_db, slack_interactor, workspace_name=workspace_name),
        SarcasticAgent(llm, action_db, slack_interactor, workspace_name=workspace_name),
        PaulGrahamAgent(llm, action_db, slack_interactor, workspace_name=workspace_name)
    ]
    for workspace_name, slack_interactor in slack_interactors.items()
}

In [12]:
            for workspace_name, slack_interactor in slack_interactors.items():
                print(f"\nFetching new messages for workspace: {slack_interactor.workspace_name}")
                data = slack_interactor.fetch_new_messages()
                threads = slack_interactor.organize_threads(data)
                print(f"Found {len(threads)} threads with new messages in {slack_interactor.workspace_name}.")

                if not slack_interactor.is_first_run:
                    results = process_threads(agents[workspace_name], threads)
                    print(f"\nChecking for due actions in {slack_interactor.workspace_name}...")
                    execute_due_actions(agents[workspace_name])
                else:
                    print(f"First run for {slack_interactor.workspace_name}. Skipping thread processing and due actions.")
                    slack_interactor.is_first_run = False



Fetching new messages for workspace: workspace1
Set conversations_oldest to 1722399936.894109
Loaded 426 old messages
Rate limited. Retrying in 1.96 seconds (attempt 1/10)
Found 1 new messages
Updated complete_conversations_agentflow.pkl with new messages
Found 1 threads with new messages in workspace1.

Processing thread in channel: agentflow
Thread timestamp: 2024-08-28 22:45:19.409008980
Last message content:
Paul, :arrow_up: is directed to you
ClaudeLLM prompt:

        You are an AI agent with the following characteristics:
        Personality: Professional and efficient
        Goal: Close the loop on open items, nudging and reminding people when necessary. Strive to not be very chatty, and only chime in efficiently.
        Username: PM Agent

        Analyze the following conversation carefully. Consider the entire thread history when making decisions. Determine if any immediate action is needed or if a delayed task should be scheduled. Consider the following:

        1. Imme

Posted reply to thread 2024-08-28 22:45:19.409008980 in channel agentflow

Executed immediate action for Sarcastic Agent: Executed immediate action: Provide sarcastic response about cofounders leaving
ClaudeLLM prompt:

        You are an AI agent with the following characteristics:
        Personality: Insightful, direct, and focused on startups and technology
        Goal: Provide thought-provoking insights on startups, technology, and innovation. Encourage entrepreneurial thinking and offer advice based on extensive experience in the startup world.
        Username: Paul Graham

        Analyze the following conversation carefully. Consider the entire thread history when making decisions. Determine if any immediate action is needed or if a delayed task should be scheduled. Consider the following:

        1. Immediate actions: Tasks that need to be done right away based on the conversation context.
        2. Delayed tasks: Any task that needs to be performed in the future, includin